In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import Levenshtein as Lv
import time

import numpy as np
import json
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Analysis of AutoPhrase results

In [2]:
fp_uniquebyyear = '../results/dblp-v10-phrases-uniquebyyear.csv'
fp_unique = '../results/dblp-v10-phrases-unique.csv'

In [3]:
# Start from year 1968 - years before had too little training data
# We only kept multi-word phrases above 0.6 and single-word above 0.8

In [4]:
# Contains the phrases unique overall (no duplicates)
un_all = pd.read_csv(fp_unique, index_col=0)
un_all = un_all[un_all['Year'] >= 1968]
un_all

,Phrase Quality,Phrase,Year
9,0.890167,time sharing,1968
10,0.610000,real time,1970
11,0.964167,pattern recognition,1972
12,0.866167,data base,1972
13,0.850167,programming languages,1972
...,...,...,...
39336,0.603183,great attention,2017
39337,0.602875,limited training,2017
39338,0.602173,public datasets,2017
39339,0.602145,existing works,2017


In [5]:
# Contains the phrases unique by year (there can be duplicates across years)
uby = pd.read_csv(fp_uniquebyyear, index_col=0)
uby = uby[uby['Year'] >= 1968]
uby = uby.dropna()
uby

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [6]:
# Only keeps the phrases that show up multiple times across years (so we can look for trends)
uby_dups = uby.copy()
uby_dups = uby_dups[uby_dups['Phrase'].duplicated(keep=False)]
uby_dups

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [7]:
uby_dups['Phrase'].value_counts()[:30]

context free               46
high level                 42
programming language       42
natural language           41
pattern recognition        41
programming languages      41
data structures            41
sufficient conditions      39
data structure             39
problem solving            39
np complete                38
general purpose            38
artificial intelligence    38
signal processing          38
linear programming         38
lower bounds               38
large scale                37
dynamic programming        37
image processing           36
information retrieval      36
data flow                  36
lower bound                36
software development       35
user interface             35
knowledge base             35
data types                 35
database systems           35
high speed                 35
software engineering       35
worst case                 35
Name: Phrase, dtype: int64

In [8]:
uby_dups[uby_dups['Phrase'] == 'image processing']

,Phrase Quality,Phrase,Year
140,0.978969,image processing,1981
172,0.984815,image processing,1982
216,0.984825,image processing,1983
289,0.969924,image processing,1984
367,0.969975,image processing,1985
455,0.976549,image processing,1986
542,0.980447,image processing,1987
692,0.983969,image processing,1988
950,0.965337,image processing,1989
1244,0.972183,image processing,1990


In [9]:
# Can look at phrases that first appear in a year, then look at future years in which they appear
data = {}

# Phrase matching/similarity for input papers (title + abstract)

In [10]:
# Given an input paper (title + abstract), extract the phrases within it and return the
# similar phrases found in the AutoPhrase results.
# Can use Levenshtein distance to find similar strings, or just use direct phrase matching

In [11]:
test_fp = '../data/arxiv/csv/2016.csv'

In [12]:
test_data = pd.read_csv(test_fp)
test_data.head()

,Title,Abstract
0,Sequential Short-Text Classification with Recu...,Recent approaches based on artificial neural n...
1,Multiresolution Recurrent Neural Networks An A...,We introduce the multiresolution recurrent neu...
2,Document Image Coding and Clustering for Scrip...,The paper introduces a new method for discrimi...
3,Tutorial on Answering Questions about Images w...,Together with the development of more accurate...
4,Building Machines That Learn and Think Like Pe...,Recent progress in artificial intelligence AI ...


In [13]:
test_data['Title'][0] + ' ' + test_data['Abstract'][0]

'Sequential Short-Text Classification with Recurrent and Convolutional  Neural Networks Recent approaches based on artificial neural networks ANNs have shownpromising results for short-text classification However many short textsoccur in sequences eg sentences in a document or utterances in a dialogand most existing ANN-based systems do not leverage the preceding short textswhen classifying a subsequent one In this work we present a model based onrecurrent neural networks and convolutional neural networks that incorporatesthe preceding short texts Our model achieves state-of-the-art results on threedifferent datasets for dialog act prediction'

In [14]:
uby

,Phrase Quality,Phrase,Year
20,0.946833,context free,1968
21,0.890167,time sharing,1968
22,0.909000,context free,1969
23,0.896000,time sharing,1969
24,0.993000,context free,1970
...,...,...,...
238718,0.600501,practical implementation,2017
238719,0.600363,next generation,2017
238720,0.600185,deep convolutional neural,2017
238721,0.600087,network nodes,2017


In [15]:
input_phrase = 'convolutional neural networks'

In [16]:
valid_ix = uby.apply(lambda x: x['Phrase'][0] == 'c' if isinstance(x['Phrase'], str) else False, axis=1)
valid_ix

20         True
21        False
22         True
23        False
24         True
          ...  
238718    False
238719    False
238720    False
238721    False
238722    False
Length: 238701, dtype: bool

In [17]:
unique_phrases = uby[valid_ix]['Phrase'].unique()
unique_phrases

array(['context free', 'computer science', 'computer graphics', ...,
       'complete characterization', 'co occurrence matrix',
       'computational approach'], dtype=object)

In [18]:
candidate = ''
dist = float('inf')
for phrase in unique_phrases:
    diff = Lv.distance(input_phrase, phrase)
    if diff < dist:
        candidate = phrase
        dist = diff

In [19]:
candidate

'convolutional neural networks'

In [20]:
dist

0

In [21]:
uby[uby['Phrase'] == 'convolutional neural networks']

,Phrase Quality,Phrase,Year
136900,0.865809,convolutional neural networks,2012
152925,0.915629,convolutional neural networks,2013
172010,0.937014,convolutional neural networks,2014
190695,0.931728,convolutional neural networks,2015
212473,0.917273,convolutional neural networks,2016
233574,0.904261,convolutional neural networks,2017


In [22]:
# Same approach, but keeping track of all candidates this time

In [23]:
candidates = []
for phrase in unique_phrases:
    dist = Lv.distance(input_phrase, phrase)
    candidates.append((dist, phrase))

In [24]:
candidates.sort()

In [25]:
candidates[:10]

[(0, 'convolutional neural networks'),
 (1, 'convolutional neural network'),
 (3, 'convolution neural network'),
 (4, 'convolutional neural network cnn'),
 (4, 'convolutional neural networks cnn'),
 (5, 'convolutional neural networks cnns'),
 (7, 'convolutional networks'),
 (8, 'convolutional network'),
 (9, 'cellular neural networks'),
 (9, 'chaotic neural networks')]

In [26]:
# Using df.apply so we can look at all phrases, not just phrases that start with the same letter

In [27]:
uby_test = uby.copy()
uby_test['Dist'] = uby_test.apply(lambda x: Lv.distance(input_phrase, x['Phrase']) if isinstance(x['Phrase'], str) else float('inf'), axis=1)
uby_test

,Phrase Quality,Phrase,Year,Dist
20,0.946833,context free,1968,22
21,0.890167,time sharing,1968,24
22,0.909000,context free,1969,22
23,0.896000,time sharing,1969,24
24,0.993000,context free,1970,22
...,...,...,...,...
238718,0.600501,practical implementation,2017,22
238719,0.600363,next generation,2017,21
238720,0.600185,deep convolutional neural,2017,14
238721,0.600087,network nodes,2017,21


In [28]:
uby_test.sort_values('Dist')[:10]

,Phrase Quality,Phrase,Year,Dist
212473,0.917273,convolutional neural networks,2016,0
233574,0.904261,convolutional neural networks,2017,0
152925,0.915629,convolutional neural networks,2013,0
190695,0.931728,convolutional neural networks,2015,0
136900,0.865809,convolutional neural networks,2012,0
172010,0.937014,convolutional neural networks,2014,0
190443,0.936880,convolutional neural network,2015,1
174069,0.898522,convolutional neural network,2014,1
212157,0.922546,convolutional neural network,2016,1
153942,0.900172,convolutional neural network,2013,1


In [29]:
# Testing on the unique overall df

In [30]:
un_all_test = un_all.copy()
un_all_test['Dist'] = un_all_test.apply(lambda x: Lv.distance(input_phrase, x['Phrase']) if isinstance(x['Phrase'], str) else float('inf'), axis=1)
un_all_test.head()

,Phrase Quality,Phrase,Year,Dist
9,0.890167,time sharing,1968,24.0
10,0.610000,real time,1970,24.0
11,0.964167,pattern recognition,1972,22.0
12,0.866167,data base,1972,24.0
13,0.850167,programming languages,1972,22.0


In [31]:
un_all_test.sort_values('Dist')[:20]

,Phrase Quality,Phrase,Year,Dist
27755,0.865809,convolutional neural networks,2012,0.0
30221,0.900172,convolutional neural network,2013,1.0
37333,0.839879,convolution neural network,2016,3.0
34364,0.918423,convolutional neural networks cnn,2015,4.0
32655,0.915458,convolutional neural network cnn,2014,4.0
32739,0.889687,deep convolutional neural networks,2014,5.0
32650,0.918479,convolutional neural networks cnns,2014,5.0
32720,0.894009,deep convolutional neural network,2014,6.0
33724,0.726266,convolutional networks,2014,7.0
34288,0.608522,convolutional network,2014,8.0


In [32]:
# Consolidating results
# Can start with the most common phrases and change phrases that are close enough (distance <= 5?)
counts = uby.groupby('Phrase').size()

In [33]:
# num_years tells us how many years the phrase has shown up in
uby['num_years'] = uby.apply(lambda x: counts[x['Phrase']], axis=1)

In [34]:
uby

,Phrase Quality,Phrase,Year,num_years
20,0.946833,context free,1968,46
21,0.890167,time sharing,1968,2
22,0.909000,context free,1969,46
23,0.896000,time sharing,1969,2
24,0.993000,context free,1970,46
...,...,...,...,...
238718,0.600501,practical implementation,2017,11
238719,0.600363,next generation,2017,17
238720,0.600185,deep convolutional neural,2017,3
238721,0.600087,network nodes,2017,10


# Model generation

In [35]:
# x: Phrase, Phrase Quality, num_years
# Phrase needs to be one hot encoded
# y: Only the year (may need to use the unique overall dataframe?)

In [36]:
# For phrase quality - standard scaler by year?
# For num_years - normalize overall

In [37]:
from sklearn.preprocessing import StandardScaler
std_pipe = Pipeline([('scale', StandardScaler())])

In [38]:
ohe_pipe = Pipeline([('one-hot', OneHotEncoder(handle_unknown='ignore'))])

In [39]:
ct = ColumnTransformer(transformers=[('ohe', ohe_pipe, ['Phrase']),
                                     ('scale', std_pipe, ['num_years']),
                                    ('keep', 'passthrough', ['Phrase Quality'])])

In [40]:
pl = Pipeline([('transform', ct), ('classifier', DecisionTreeClassifier())])

In [41]:
# Set random seed
X_train, X_test, y_train, y_test = train_test_split(uby[['Phrase', 'num_years', 'Phrase Quality']], 
                                                    uby['Year'],
                                                    random_state=1)

In [42]:
pl.fit(X_train, y_train)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ohe',
                                                  Pipeline(steps=[('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Phrase']),
                                                 ('scale',
                                                  Pipeline(steps=[('scale',
                                                                   StandardScaler())]),
                                                  ['num_years']),
                                                 ('keep', 'passthrough',
                                                  ['Phrase Quality'])])),
                ('classifier', DecisionTreeClassifier())])

In [43]:
# Mean accuracy - 8% accurate
pl.score(X_test, y_test)

0.08066894563978819

In [44]:
X_test['Predicted Year'] = pl.predict(X_test)

In [45]:
X_test['Year'] = y_test

In [46]:
X_test

,Phrase,num_years,Phrase Quality,Predicted Year,Year
198146,band selection,11,0.831558,2009,2015
152976,mixed criticality,7,0.914890,2015,2013
200739,rna seq data,7,0.807498,2012,2015
238274,final result,1,0.642976,2016,2017
142017,presidential,4,0.806453,2015,2012
...,...,...,...,...,...
64293,treatment planning,12,0.792733,2011,2007
237679,default mode,10,0.696536,2009,2017
198475,user behaviour,12,0.828791,2015,2015
5578,temporal reasoning,17,0.905742,2009,1996


In [47]:
X_test['Abs Year Diff'] = abs(X_test['Year'] - X_test['Predicted Year'])

In [48]:
X_test

,Phrase,num_years,Phrase Quality,Predicted Year,Year,Abs Year Diff
198146,band selection,11,0.831558,2009,2015,6
152976,mixed criticality,7,0.914890,2015,2013,2
200739,rna seq data,7,0.807498,2012,2015,3
238274,final result,1,0.642976,2016,2017,1
142017,presidential,4,0.806453,2015,2012,3
...,...,...,...,...,...,...
64293,treatment planning,12,0.792733,2011,2007,4
237679,default mode,10,0.696536,2009,2017,8
198475,user behaviour,12,0.828791,2015,2015,0
5578,temporal reasoning,17,0.905742,2009,1996,13


In [49]:
X_test['Abs Year Diff'].mean()

4.817045378376567

In [50]:
# THINGS TO TEST:
# Using the unique by year dataframe, then only keeping one instance of duplicate phrases
# but replace the year with the average (or median) of the years
# Normalizing the numeric features

# Try using phrasal segmentation model to run on a single paper title + abstract
# Or a single paper full paper text

# Try grouping by papers by a range of years (maybe 5)

In [51]:
X_train[X_train['Phrase']=='convolutional neural networks']

,Phrase,num_years,Phrase Quality
212473,convolutional neural networks,6,0.917273
152925,convolutional neural networks,6,0.915629
190695,convolutional neural networks,6,0.931728
136900,convolutional neural networks,6,0.865809


In [52]:
test = pd.DataFrame([[1.0, 'convolutional neural networks', 2005, 6]], columns=['Phrase Quality', 'Phrase', 'Year', 'num_years'])
test

,Phrase Quality,Phrase,Year,num_years
0,1.0,convolutional neural networks,2005,6


In [53]:
pl.predict(test[['Phrase', 'num_years', 'Phrase Quality']])

array([1991], dtype=int64)

In [54]:
uby[uby['Phrase'] == 'convolutional neural networks']

,Phrase Quality,Phrase,Year,num_years
136900,0.865809,convolutional neural networks,2012,6
152925,0.915629,convolutional neural networks,2013,6
172010,0.937014,convolutional neural networks,2014,6
190695,0.931728,convolutional neural networks,2015,6
212473,0.917273,convolutional neural networks,2016,6
233574,0.904261,convolutional neural networks,2017,6


In [55]:
test = pd.DataFrame([[1.0, 'artificial intelligence', 2005, 6]], columns=['Phrase Quality', 'Phrase', 'Year', 'num_years'])
test

,Phrase Quality,Phrase,Year,num_years
0,1.0,artificial intelligence,2005,6


In [56]:
pl.predict(test[['Phrase', 'num_years', 'Phrase Quality']])

array([1991], dtype=int64)

In [57]:
#model.fit(X_train, y_train)

In [58]:
# Can try using correlation between phrases
# Don't have to use a model, just calculate the correlation of input phrases against phrases by year

In [59]:
# Using string similarity
# https://towardsdatascience.com/calculating-string-similarity-in-python-276e18a7d33a
